# Step 1 to create GOES data timeseries

- Creates files with lists of all GOES files in 1 day

- Run this before creating GOES timeseries data

- Set start date

- Set number of days to increment through (does 1 day at a time)

This primarily supports creating the historical GOES data. This has to access 288 files per day, and with all those requests, there is risk of failure. Having a static list of filenames to go through was a quick solution to be able to make sure we started again at the same place we ended when the program died.

You can check what data is available here: [https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-MCMIPC/]()



In [ ]:
#create goes data
from goes2go import GOES
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#import configuration location and filepath
from myconfig import *
output_path = output_path_goes_fname

In [ ]:
now = datetime.now()
start_year = int(now.strftime("%Y"))
start_month = int(now.strftime("%m"))
start_day = int(now.strftime("%d"))
start_hour = int(now.strftime("%H"))
print(start_year, start_month, start_day,start_hour)

In [ ]:
# Open the GOES-R image
G = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')  #ABI-L2-DMWVC ABI-L2-DMWC not at all points #
for incr in range(number_days_to_process,-1,-1):
    date_start = datetime(start_year, start_month, start_day) - timedelta(days=incr)
    start_time = date_start.strftime("%Y%m%d")
    if incr == 0:
        date_end = now #date_start + timedelta(days=1)  
        end_time = (date_start + timedelta(days=1)).strftime("%Y%m%d")
    else:
        date_end = date_start + timedelta(days=1)  
        end_time = date_end.strftime("%Y%m%d")
    
    print(incr,start_time,end_time)
    # Produce a pandas DataFrame of the available files in a time range
    df = G.df(start=date_start, end=date_end)
    print(incr,len(df), df.file[0])
    # Save DataFrame to a CSV file
    #end_time = date_end.strftime("%Y%m%d")
    files_fname = output_path+'goes_filenames_test_'+start_time+'-'+end_time+'.csv'
    print('out:',files_fname)
    df.to_csv(files_fname)
